In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 데이터 로드 및 전처리
df = pd.read_csv("하이닉스 power (3).csv", thousands=',', encoding='cp949')
df['next_day_return'] = (df['종가'].shift(-1) - df['종가']) / df['종가'] * 100
df['target'] = df['next_day_return'].apply(lambda x: 1 if x > 0.25 else 0)
df.dropna(inplace=True)

X = df.drop(['날짜', 'target', 'next_day_return'], axis=1)
y = df['target']

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# 그라디언트 부스팅 모델을 위한 하이퍼파라미터 그리드 정의
param_grid_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [4,6,8],
    'min_samples_split': [2, 5,7],
    'min_samples_leaf': [1, 3]
}

# GridSearchCV 객체 초기화 및 학습
grid_search_gb = GridSearchCV(GradientBoostingClassifier(random_state=1), param_grid_gb, refit=True, verbose=2, cv=5, n_jobs=-1)
grid_search_gb.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 모델 출력
print(f"Best parameters: {grid_search_gb.best_params_}")
best_gb_model = grid_search_gb.best_estimator_

# 최적의 모델로 테스트 데이터 예측
y_pred_gb = best_gb_model.predict(X_test)

# 예측 결과에 대한 평가 지표 출력
accuracy_gb = accuracy_score(y_test, y_pred_gb)
precision_gb = precision_score(y_test, y_pred_gb, average='macro')
recall_gb = recall_score(y_test, y_pred_gb, average='macro')
f1_gb = f1_score(y_test, y_pred_gb, average='macro')

print(f"Accuracy (GB): {accuracy_gb:.4f}")
print(f"Precision (GB): {precision_gb:.4f}")
print(f"Recall (GB): {recall_gb:.4f}")
print(f"F1 Score (GB): {f1_gb:.4f}")